In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

print(f"syft version: {sy.__version__}")

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")

In [ ]:
high_client = server_high.login(email="info@openmined.org", password="changethis")

In [ ]:
low_client.worker_pools.get_all()

In [ ]:
assert len(high_client.worker_pools.get_all()) == 1
assert len(low_client.worker_pools.get_all()) == 1

In [ ]:
def launch_worker_pool(client, pool_name):
    if pool_name not in [x.name for x in client.worker_pools]:
        external_registry = test_settings.get("external_registry", default="docker.io")
        worker_docker_tag = f"openmined/bigquery:{sy.__version__}"
        result = client.api.services.worker_image.submit(
            worker_config=sy.PrebuiltWorkerConfig(
                tag=f"{external_registry}/{worker_docker_tag}"
            )
        )
        worker_image = client.images.get_all()[1]
        result = client.api.services.image_registry.add(external_registry)
        result = client.api.services.worker_pool.launch(
            pool_name=pool_name,
            image_uid=worker_image.id,
            num_workers=1,
        )
        return result
    else:
        print("Pool already exists")

In [ ]:
pool_name = "bigquery-pool"

In [ ]:
launch_worker_pool(high_client, pool_name)

In [ ]:
launch_worker_pool(low_client, pool_name)

In [ ]:
# result = high_client.worker_pools.scale(number=5, pool_name=pool_name)
# result

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2
assert len(low_client.worker_pools.get_all()) == 2

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

In [ ]:
low_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
high_client.settings.allow_guest_signup(enable=False)
low_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert (
    len(low_client.api.services.user.get_all()) == 2
), "Only DS and Admin should be at low side"
assert (
    len(high_client.api.services.user.get_all()) == 1
), "Only Admin should be at high side"

In [ ]:
server_high.land()
server_low.land()